In [1]:
# imports necessary libraries
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np

In [2]:
# Read csvs

train_df = gpd.read_file('train.geojson', index_col=0)
test_df = gpd.read_file('test.geojson', index_col=0)


In [3]:
# Filtering columns on sets
le = LabelEncoder()

def get_x(df):
    x = pd.DataFrame()

    # status dates
    for i in range(1,6):
        x['change_status_date{}'.format(i)] = le.fit_transform(df['change_status_date{}'.format(i)])

    # dates
    print("status ok")
    for i in range(1,6):
        x['year{}'.format(i)] = le.fit_transform(df['date{}'.format(i)].transform(lambda x: x[-4:-1]+x[-1]))
    for i in range(1,6):
        x['month{}'.format(i)] = le.fit_transform(df['date{}'.format(i)].transform(lambda x: x[-7:-5]))

    # urban types
    print("date ok")
    x['urban_types'] = le.fit_transform(df['urban_types'])
    print("urban ok")

    # geography types
    geography_types_map = {'River': 0,'Sparse Forest': 1,'Grass Land': 2,'Farms': 3,'Lakes': 4,'Barren Land': 5,'Coastal': 6,'Dense Forest': 7,'None': 8,'Hills': 9,'Desert': 10,'Snow': 11}
    for geography_type in geography_types_map:
        x[geography_type] = 0

    # for i, geography_features in enumerate(df.loc[:, ('geography_types')]):
    #     features = geography_features.split(',')
    #     for feature in features:
    #         x[feature][i] = 1

    # geometry features
    # area
    print("okk")
    x['area'] = df[['geometry']].area
    # perimeter
    x['perimeter'] = df[['geometry']].length
    # centroid of the polygon
    x['x_centroid'] = df[['geometry']].centroid.x
    x['y_centroid'] = df[['geometry']].centroid.y
    # length on the x and y axis
    x['delta_x'] = df[['geometry']].bounds.maxx.subtract(df[['geometry']].bounds.minx)
    x['delta_y'] = df[['geometry']].bounds.maxy.subtract(df[['geometry']].bounds.miny)
    # angle of the diagonal of the rectangle made by delta_x and delta_y
    x['angle_diago'] = (x['delta_x'].div((x['delta_x'].apply(lambda x: x**2) + x['delta_y'].apply(lambda x: x**2)).apply(np.sqrt))).apply(np.arccos)

    return x

In [4]:
x_train = get_x(train_df)
x_test = get_x(test_df)
print(x_train.shape)
print(x_test.shape)

status ok
date ok
urban ok
okk
status ok
date ok
urban ok
okk
(310006, 35)
(121704, 35)


In [5]:
change_type_map = {'Demolition': np.array([1,0,0,0,0,0]), 'Road': np.array([0,1,0,0,0,0]), 'Residential': np.array([0,0,1,0,0,0]), 'Commercial': np.array([0,0,0,1,0,0]), 'Industrial': np.array([0,0,0,0,1,0]),
                   'Mega Projects': np.array([0,0,0,0,0,1])}

train_y = train_df['change_type'].apply(lambda x: change_type_map[x])

In [16]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

s=[]
# for n in range(3,35,2):
#     pca = PCA(n_components=n)
#     pca.fit(x_train)
#     s.append(pca.score(x_train))
pca = PCA(n_components=15)
x_train = pca.fit_transform(x_train)
# plt.plot(list(range(3,35,2)),s)

In [17]:
import tensorflow as tf
from keras import layers
from keras import regularizers

modele = tf.keras.Sequential([
layers.Dense(50, activation = 'relu', input_shape = [x_train.shape[1]]),
layers.Dropout(0.4),
layers.Dense(30, activity_regularizer=regularizers.l2(1e-3), kernel_regularizer=regularizers.l1_l2(l1=1e-3, l2=1e-3), bias_regularizer=regularizers.l2(1e-3), activation = 'relu'),
layers.Dropout(0.3),
layers.Dense(6, activation = 'softmax')])

In [18]:
modele.compile(loss = 'categorical_crossentropy',
optimizer = tf.keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95, epsilon=1e-07, name="Adadelta"),
metrics = ['accuracy'])

In [19]:
train_y = np.stack(np.array(train_y))

In [20]:
modele.fit(x_train, train_y, epochs = 30, validation_split = 0.2, batch_size=128)

Epoch 1/30
1938/1938 [==============================] - 5s 2ms/step - loss: 1.3005 - accuracy: 0.5753 - val_loss: 0.8768 - val_accuracy: 0.6382
Epoch 2/30
1938/1938 [==============================] - 4s 2ms/step - loss: 0.8560 - accuracy: 0.6518 - val_loss: 0.8500 - val_accuracy: 0.6373
Epoch 3/30
1938/1938 [==============================] - 4s 2ms/step - loss: 0.8326 - accuracy: 0.6595 - val_loss: 0.8407 - val_accuracy: 0.6384
Epoch 4/30
1938/1938 [==============================] - 4s 2ms/step - loss: 0.8252 - accuracy: 0.6628 - val_loss: 0.8357 - val_accuracy: 0.6529
Epoch 5/30
1938/1938 [==============================] - 4s 2ms/step - loss: 0.8202 - accuracy: 0.6649 - val_loss: 0.8358 - val_accuracy: 0.6412
Epoch 6/30
1938/1938 [==============================] - 4s 2ms/step - loss: 0.8165 - accuracy: 0.6663 - val_loss: 0.8357 - val_accuracy: 0.6458
Epoch 7/30
1938/1938 [==============================] - 4s 2ms/step - loss: 0.8131 - accuracy: 0.6670 - val_loss: 0.8475 - val_accuracy:

In [21]:
pred = modele.predict(x_test)

ValueError: in user code:

    File "C:\Users\remib\anaconda3\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\remib\anaconda3\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\remib\anaconda3\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\remib\anaconda3\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\remib\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\remib\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {input_index}

In [ ]:
pred_submit = []
for i in range(len(pred)):
    max_proba  = max(pred[i])
    prediction = list(pred[i]).index(max_proba)
    pred_submit.append(prediction)

pred_df = pd.DataFrame(pred_submit, columns=['change_type'])
pred_df.to_csv("knn_sample_submission.csv", index=True, index_label='Id')